<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/NetGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import torch.optim as optim
import time


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, 462)
        self.fc2 = nn.Linear(462, 462*5)
        self.fc3 = nn.Linear(462*5, 231*461)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = MyNet().to(device)

# for n in range(1,151):
#     PATH = 'Data/Train1-150/' + str(n)
f = open("1000", 'rb')
data = pickle.load(f)

criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=1e-4)

for i in range(10):
    print(i)

    optimizer.zero_grad()

    input = torch.from_numpy(data[i][1]).float().to(device)
    true_y = torch.FloatTensor(data[i][0]).to(device)
    output = net(input)

    loss = criterion(output, true_y)
    RMS = (loss.item()/106491)**0.5
    print(f'Epoch {i}: {RMS}')

    loss.backward()
    optimizer.step()


PATH = 'model.pth'
torch.save(net.state_dict(), PATH)


0
Epoch 0: 0.5765968016330589
1
Epoch 1: 0.5749221584942192
2
Epoch 2: 0.5723901932523436
3
Epoch 3: 0.5700329470693514
4
Epoch 4: 0.5683596164234875
5
Epoch 5: 0.5650596966231591
6
Epoch 6: 0.56261055533887
7
Epoch 7: 0.5599362666625427
8
Epoch 8: 0.5559735552180958
9
Epoch 9: 0.5527584664782474
